### Deliverable 1: Preprocessing the Data for a Neural Network

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

In [16]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [17]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN","NAME","ASK_AMT","INCOME_AMT"],inplace=True)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,N,1
1,T3,Independent,C2000,Preservation,Co-operative,1,N,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,N,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,N,1
4,T3,Independent,C1000,Heathcare,Trust,1,N,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,N,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,N,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,N,0
34297,T5,Independent,C3000,ProductDev,Association,1,N,1


In [18]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
STATUS                     2
SPECIAL_CONSIDERATIONS     2
IS_SUCCESSFUL              2
dtype: int64

In [19]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'SPECIAL_CONSIDERATIONS']

In [20]:
application_df.AFFILIATION.value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [21]:
# delete "Other" in AFFILIATION
application_df = application_df[(application_df["AFFILIATION"] != "Other") ]

In [22]:
application_df.USE_CASE.value_counts()

Preservation     28092
ProductDev        5670
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [23]:
# delete "Other" in USE_CASE
application_df = application_df[(application_df["USE_CASE"] != "Other") ]

In [24]:
application_df.ORGANIZATION.value_counts()

Trust           23511
Association     10254
Co-operative      486
Corporation        41
Name: ORGANIZATION, dtype: int64

In [25]:
application_df.SPECIAL_CONSIDERATIONS.value_counts()

N    34265
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [26]:
# Look at APPLICATION_TYPE value counts for binning
application_df.APPLICATION_TYPE.value_counts()

T3     27032
T4      1542
T6      1215
T5      1173
T19     1065
T8       736
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [29]:
APPLICATION_TYPE_counts =application_df.APPLICATION_TYPE.value_counts()

# Determine which values to replace if counts are less than ...?
replace_application = list(APPLICATION_TYPE_counts[APPLICATION_TYPE_counts < 100].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27032
T4        1542
T6        1215
T5        1173
T19       1065
T8         736
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [32]:
application_df.CLASSIFICATION.value_counts()


C1000    17320
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1728        1
C2500        1
C1900        1
C4200        1
C1236        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [33]:
classification_count = application_df.CLASSIFICATION.value_counts()
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count<1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17320
C2000     6074
C1200     4837
Other     2260
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [34]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'SPECIAL_CONSIDERATIONS']

In [35]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,...,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [36]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True,right_index=True).drop(application_cat,1)
application_df.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,1,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [37]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [38]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 2: Compile, Train and Evaluate the Model

In [39]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 90
hidden_nodes_layer3 = 70

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               3300      
_________________________________________________________________
dense_1 (Dense)              (None, 90)                9090      
_________________________________________________________________
dense_2 (Dense)              (None, 70)                6370      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 71        
Total params: 18,831
Trainable params: 18,831
Non-trainable params: 0
_________________________________________________________________


In [40]:
# Compile the model

#keras.losses.SparseCategoricalCrossentropy()
#metrics=[keras.metrics.SparseCategoricalAccuracy()]
#"binary_crossentropy"
#metrics=["accuracy"]

import keras
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [41]:
# Train the model
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)

checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

fit_model = nn.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
 81/804 [==>...........................] - ETA: 0s - loss: 0.6870 - accuracy: 0.5559  
Epoch 00001: saving model to checkpoints\weights.01.hdf5
200/804 [======>.......................] - ETA: 0s - loss: 0.6853 - accuracy: 0.5647
Epoch 00001: saving model to checkpoints\weights.01.hdf5
264/804 [========>.....................] - ETA: 0s - loss: 0.6843 - accuracy: 0.5690
Epoch 00001: saving model to checkpoints\weights.01.hdf5
330/804 [===========>..................] - ETA: 0s - loss: 0.6835 - accuracy: 0.5696
Epoch 00001: saving model to checkpoints\weights.01.hdf5
480/804 [================>.............] - ETA: 0s - loss: 0.6819 - accuracy: 0.5724
Epoch 00001: saving model to checkpoints\weights.01.hdf5
547/804 [===================>..........] - ETA: 0s - loss: 0.6825 - accuracy: 0.5700
Epoch 00001: saving model to checkpoints\weights.01.hdf5
685/804 [========================>.....] - ETA: 0s - loss: 0.6826 - accuracy: 0.5710
Epoch 00001: saving model to checkpoints\weights.

571/804 [====================>.........] - ETA: 0s - loss: 0.6753 - accuracy: 0.5854
Epoch 00007: saving model to checkpoints\weights.07.hdf5
633/804 [======================>.......] - ETA: 0s - loss: 0.6757 - accuracy: 0.5843
Epoch 00007: saving model to checkpoints\weights.07.hdf5
703/804 [=========================>....] - ETA: 0s - loss: 0.6764 - accuracy: 0.5819
Epoch 00007: saving model to checkpoints\weights.07.hdf5
804/804 [==============================] - 1s 725us/step - loss: 0.6770 - accuracy: 0.5803
Epoch 8/100
  1/804 [..............................] - ETA: 0s - loss: 0.6656 - accuracy: 0.6250
Epoch 00008: saving model to checkpoints\weights.08.hdf5
139/804 [====>.........................] - ETA: 0s - loss: 0.6764 - accuracy: 0.5814
Epoch 00008: saving model to checkpoints\weights.08.hdf5
202/804 [======>.......................] - ETA: 0s - loss: 0.6775 - accuracy: 0.5769
Epoch 00008: saving model to checkpoints\weights.08.hdf5
349/804 [============>.................] - ET

138/804 [====>.........................] - ETA: 0s - loss: 0.6753 - accuracy: 0.5811
Epoch 00014: saving model to checkpoints\weights.14.hdf5
196/804 [======>.......................] - ETA: 0s - loss: 0.6736 - accuracy: 0.5875
Epoch 00014: saving model to checkpoints\weights.14.hdf5
326/804 [===========>..................] - ETA: 0s - loss: 0.6739 - accuracy: 0.5854
Epoch 00014: saving model to checkpoints\weights.14.hdf5
387/804 [=============>................] - ETA: 0s - loss: 0.6740 - accuracy: 0.5858
Epoch 00014: saving model to checkpoints\weights.14.hdf5
525/804 [==================>...........] - ETA: 0s - loss: 0.6746 - accuracy: 0.5824
Epoch 00014: saving model to checkpoints\weights.14.hdf5
586/804 [====================>.........] - ETA: 0s - loss: 0.6749 - accuracy: 0.5818
Epoch 00014: saving model to checkpoints\weights.14.hdf5
724/804 [==========================>...] - ETA: 0s - loss: 0.6756 - accuracy: 0.5816
Epoch 00014: saving model to checkpoints\weights.14.hdf5
804/80

571/804 [====================>.........] - ETA: 0s - loss: 0.6752 - accuracy: 0.5824
Epoch 00020: saving model to checkpoints\weights.20.hdf5
718/804 [=========================>....] - ETA: 0s - loss: 0.6750 - accuracy: 0.5824
Epoch 00020: saving model to checkpoints\weights.20.hdf5
804/804 [==============================] - 1s 763us/step - loss: 0.6751 - accuracy: 0.5813
Epoch 21/100
  1/804 [..............................] - ETA: 0s - loss: 0.7006 - accuracy: 0.4688
Epoch 00021: saving model to checkpoints\weights.21.hdf5
 66/804 [=>............................] - ETA: 0s - loss: 0.6706 - accuracy: 0.5956
Epoch 00021: saving model to checkpoints\weights.21.hdf5
211/804 [======>.......................] - ETA: 0s - loss: 0.6729 - accuracy: 0.5872
Epoch 00021: saving model to checkpoints\weights.21.hdf5
276/804 [=========>....................] - ETA: 0s - loss: 0.6740 - accuracy: 0.5840
Epoch 00021: saving model to checkpoints\weights.21.hdf5
340/804 [===========>..................] - E

132/804 [===>..........................] - ETA: 0s - loss: 0.6704 - accuracy: 0.5909
Epoch 00027: saving model to checkpoints\weights.27.hdf5
265/804 [========>.....................] - ETA: 0s - loss: 0.6755 - accuracy: 0.5777
Epoch 00027: saving model to checkpoints\weights.27.hdf5
384/804 [=============>................] - ETA: 0s - loss: 0.6737 - accuracy: 0.5826
Epoch 00027: saving model to checkpoints\weights.27.hdf5
439/804 [===============>..............] - ETA: 0s - loss: 0.6740 - accuracy: 0.5835
Epoch 00027: saving model to checkpoints\weights.27.hdf5
565/804 [====================>.........] - ETA: 0s - loss: 0.6733 - accuracy: 0.5845
Epoch 00027: saving model to checkpoints\weights.27.hdf5
682/804 [========================>.....] - ETA: 0s - loss: 0.6734 - accuracy: 0.5842
Epoch 00027: saving model to checkpoints\weights.27.hdf5
775/804 [===========================>..] - ETA: 0s - loss: 0.6739 - accuracy: 0.5836
Epoch 00027: saving model to checkpoints\weights.27.hdf5
804/80

...

804/804 [==============================] - 1s 742us/step - loss: 0.6723 - accuracy: 0.5846
Epoch 73/100
  1/804 [..............................] - ETA: 0s - loss: 0.6293 - accuracy: 0.7500
Epoch 00073: saving model to checkpoints\weights.73.hdf5
 70/804 [=>............................] - ETA: 0s - loss: 0.6722 - accuracy: 0.5888
Epoch 00073: saving model to checkpoints\weights.73.hdf5
135/804 [====>.........................] - ETA: 0s - loss: 0.6711 - accuracy: 0.5905
Epoch 00073: saving model to checkpoints\weights.73.hdf5
294/804 [=========>....................] - ETA: 0s - loss: 0.6727 - accuracy: 0.5855
Epoch 00073: saving model to checkpoints\weights.73.hdf5
364/804 [============>.................] - ETA: 0s - loss: 0.6732 - accuracy: 0.5834
Epoch 00073: saving model to checkpoints\weights.73.hdf5
510/804 [==================>...........] - ETA: 0s - loss: 0.6734 - accuracy: 0.5829
Epoch 00073: saving model to checkpoints\weights.73.hdf5
577/804 [====================>.........] - E

351/804 [============>.................] - ETA: 0s - loss: 0.6709 - accuracy: 0.5840
Epoch 00079: saving model to checkpoints\weights.79.hdf5
420/804 [==============>...............] - ETA: 0s - loss: 0.6711 - accuracy: 0.5849
Epoch 00079: saving model to checkpoints\weights.79.hdf5
566/804 [====================>.........] - ETA: 0s - loss: 0.6722 - accuracy: 0.5848
Epoch 00079: saving model to checkpoints\weights.79.hdf5
635/804 [======================>.......] - ETA: 0s - loss: 0.6726 - accuracy: 0.5840
Epoch 00079: saving model to checkpoints\weights.79.hdf5
773/804 [===========================>..] - ETA: 0s - loss: 0.6729 - accuracy: 0.5835
Epoch 00079: saving model to checkpoints\weights.79.hdf5
804/804 [==============================] - 1s 720us/step - loss: 0.6721 - accuracy: 0.5846
Epoch 80/100
 82/804 [==>...........................] - ETA: 0s - loss: 0.6721 - accuracy: 0.5804
Epoch 00080: saving model to checkpoints\weights.80.hdf5
146/804 [====>.........................] - E

804/804 [==============================] - 1s 851us/step - loss: 0.6722 - accuracy: 0.5842
Epoch 86/100
  1/804 [..............................] - ETA: 1s - loss: 0.6356 - accuracy: 0.6875
Epoch 00086: saving model to checkpoints\weights.86.hdf5
116/804 [===>..........................] - ETA: 0s - loss: 0.6644 - accuracy: 0.5978
Epoch 00086: saving model to checkpoints\weights.86.hdf5
256/804 [========>.....................] - ETA: 0s - loss: 0.6664 - accuracy: 0.5958
Epoch 00086: saving model to checkpoints\weights.86.hdf5
316/804 [==========>...................] - ETA: 0s - loss: 0.6675 - accuracy: 0.5939
Epoch 00086: saving model to checkpoints\weights.86.hdf5
386/804 [=============>................] - ETA: 0s - loss: 0.6707 - accuracy: 0.5878
Epoch 00086: saving model to checkpoints\weights.86.hdf5
541/804 [===================>..........] - ETA: 0s - loss: 0.6707 - accuracy: 0.5873
Epoch 00086: saving model to checkpoints\weights.86.hdf5
599/804 [=====================>........] - E

358/804 [============>.................] - ETA: 0s - loss: 0.6725 - accuracy: 0.5842
Epoch 00092: saving model to checkpoints\weights.92.hdf5
514/804 [==================>...........] - ETA: 0s - loss: 0.6716 - accuracy: 0.5852
Epoch 00092: saving model to checkpoints\weights.92.hdf5
578/804 [====================>.........] - ETA: 0s - loss: 0.6711 - accuracy: 0.5872
Epoch 00092: saving model to checkpoints\weights.92.hdf5
721/804 [=========================>....] - ETA: 0s - loss: 0.6715 - accuracy: 0.5856
Epoch 00092: saving model to checkpoints\weights.92.hdf5
804/804 [==============================] - 1s 705us/step - loss: 0.6718 - accuracy: 0.5849
Epoch 93/100
  1/804 [..............................] - ETA: 0s - loss: 0.6864 - accuracy: 0.5625
Epoch 00093: saving model to checkpoints\weights.93.hdf5
 67/804 [=>............................] - ETA: 0s - loss: 0.6806 - accuracy: 0.5718
Epoch 00093: saving model to checkpoints\weights.93.hdf5
218/804 [=======>......................] - E

  1/804 [..............................] - ETA: 0s - loss: 0.6353 - accuracy: 0.7500
Epoch 00099: saving model to checkpoints\weights.99.hdf5
 62/804 [=>............................] - ETA: 0s - loss: 0.6704 - accuracy: 0.5953
Epoch 00099: saving model to checkpoints\weights.99.hdf5
208/804 [======>.......................] - ETA: 0s - loss: 0.6699 - accuracy: 0.5925
Epoch 00099: saving model to checkpoints\weights.99.hdf5
276/804 [=========>....................] - ETA: 0s - loss: 0.6695 - accuracy: 0.5909
Epoch 00099: saving model to checkpoints\weights.99.hdf5
341/804 [===========>..................] - ETA: 0s - loss: 0.6698 - accuracy: 0.5911
Epoch 00099: saving model to checkpoints\weights.99.hdf5
488/804 [=================>............] - ETA: 0s - loss: 0.6707 - accuracy: 0.5863
Epoch 00099: saving model to checkpoints\weights.99.hdf5
557/804 [===================>..........] - ETA: 0s - loss: 0.6707 - accuracy: 0.5871
Epoch 00099: saving model to checkpoints\weights.99.hdf5
698/80

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 15.5994 - accuracy: 0.4998
Loss: 15.599366188049316, Accuracy: 0.49976667761802673


In [538]:
nn.save("AlphabetSoupCharity_Optimization.h5")